In [ ]:
import numpy as np
from numpy.linalg import norm
from fpylll import FPLLL, SVP, CVP
from copy import copy
import math
from svp import __decision_svp
from lattice_generator import generate_random_instance, generate_hard_instance, reduced_basis

np.random.seed(1337)
FPLLL.set_random_seed(1337)

In [ ]:
n, b= 14, 32
X = generate_random_instance(b, n)

# n, p, r= 17, 97, 3
# X = generate_hard_instance(n, p, r)

A, B = reduced_basis(X, n)

X=copy(A)
SVP.shortest_vector(A)
s, l = A[0], norm(A[0])
print('------------------------------------')
print('fpylll solution:')
print([int(x) for x in s],'\n Norm:', l)
# print([int(x) for x in inv(B)@s])

print('------------------------------------')
print('my solution:')
C = 1															#Working fine, but C~20: 2*e*pi
num_samples=int(2**(C*n))*int(math.log2(n))
s, _l = __decision_svp(B, n, l, l, num_samples)

print([-int(x) for x in s],'\n Norm:', _l)

print('------------------------------------')
print('verify solution:')
# print(X)
# print('coefficients vector:  ',[int(x) for x in pinv(B)@s])
# print('reconstructed vector: ' ,list(B@[int(x) for x in pinv(B)@s]))

v0 = CVP.closest_vector(X, tuple([int(x) for x in s]))
# print('Verify with CVP:      ',[int(x) for x in v0],'\n norm:',norm(v0))
print('Verdict:', bool(abs(norm(v0)-_l)<1e-3), ', Comparing solutions:', bool(l<=_l+1e-6))

------------------------------------
fpylll solution:
[138858514, 1386531463, 1395983580, 1102869228, 2373372017, -83745651, -89447151, 306420357, 1279104478, 611527299, 1534556608, -884584334, -281569383, 109045096] 
 Norm: 4010890630.299641
------------------------------------
my solution:
[-138858514, -1386531463, -1395983580, -1102869228, -2373372017, 83745651, 89447151, -306420357, -1279104478, -611527299, -1534556608, 884584334, 281569383, -109045096] 
 Norm: 4010890630.299641
------------------------------------
verify solution:
Verdict: True , Comparing solutions: True
